<h1>Recent Uncleared Sidewalk Violations, 2023-2024</h1>
11 January 2024

This analysis looks at 311 complaints filed as "Snow – Uncleared Sidewalk Complaint," or SWSNOREM in the current (2023-2024) snow season.<br>
<br>
My analysis steps:
<ol>
<li><a href="#read">Read Data</a>
<li><a href="#ward">By Ward, 1-Year and 5-Year</a>
<li>Ward X
</ol>

<div style="color:red;">
    <h3>Preliminary Findings</h3>
<ul>
    <li>...
</ul>
    </div>

<a name="read"></a>
# 1. Read Data

In [1]:
import pandas as pd
import requests
#import datetime as dt #would only need this if I manipulated dates post-API data retrieval

In [20]:
base_url = "https://data.cityofchicago.org/resource/v6vf-nfxy.json"
select = "SR_NUMBER, SR_SHORT_CODE, CREATED_DATE, STREET_ADDRESS, COMMUNITY_AREA, WARD, STATUS, ORIGIN, CLOSED_DATE, LATITUDE, LONGITUDE, SR_TYPE"
# where = "SR_SHORT_CODE = 'SWSNOREM' AND CREATED_DATE >= '2023-07-01 00:00:00'"
where = "SR_SHORT_CODE = 'SWSNOREM'"
limit = 99999

url = f"{base_url}?$SELECT={select}&$WHERE={where}&$LIMIT={limit}"
print (url)

https://data.cityofchicago.org/resource/v6vf-nfxy.json?$SELECT=SR_NUMBER, SR_SHORT_CODE, CREATED_DATE, STREET_ADDRESS, COMMUNITY_AREA, WARD, STATUS, ORIGIN, CLOSED_DATE, LATITUDE, LONGITUDE, SR_TYPE&$WHERE=SR_SHORT_CODE = 'SWSNOREM'&$LIMIT=99999


In [25]:
response = requests.get(url)
data = response.json()
print (response)

<Response [200]>


In [26]:
df=pd.DataFrame(data)
df.head()

,SR_NUMBER,SR_SHORT_CODE,CREATED_DATE,STATUS,ORIGIN,CLOSED_DATE,SR_TYPE,STREET_ADDRESS,COMMUNITY_AREA,WARD,LATITUDE,LONGITUDE
0,SR19-02929866,SWSNOREM,2019-11-12T09:31:20.000,Completed,Internet,2019-11-21T09:15:03.000,Snow – Uncleared Sidewalk Complaint,NaN,NaN,NaN,NaN,NaN
1,SR20-05649092,SWSNOREM,2020-12-17T13:24:37.000,Completed,Phone Call,2020-12-18T15:12:51.000,Snow – Uncleared Sidewalk Complaint,5200 S BLACKSTONE AVE,41,4,41.800909458,-87.590514593
2,SR21-00001364,SWSNOREM,2021-01-01T11:40:33.000,Completed,Mobile Device,2021-01-04T14:33:44.000,Snow – Uncleared Sidewalk Complaint,66 E CHESTNUT ST,8,42,41.89839859,-87.625722581
3,SR21-00000774,SWSNOREM,2021-01-01T09:11:15.000,Completed,Internet,2021-01-06T13:37:58.000,Snow – Uncleared Sidewalk Complaint,2648 N WHIPPLE ST,22,32,41.930232612,-87.703705561
4,SR20-05723378,SWSNOREM,2020-12-31T07:19:37.000,Completed,Mobile Device,2021-01-08T15:30:57.000,Snow – Uncleared Sidewalk Complaint,5015 N SPRINGFIELD AVE,14,39,41.972438885,-87.725429908


### parse dates 

In [27]:
def get_season(month, year):
# Input:
#   month: numeric month of the year (1-12)
#   year (int): 4 digit year
# Returns:
#   season in yy-yy format. E.g. 22-23 means winter 2022-2023, or July 2022 to June 2023.

    if month >= 7:
        return f"{str(year)[-4:]}-{str(year + 1)[-4:]}"
    else:
        return f"{str(year - 1)[-4:]}-{str(year)[-4:]}"

In [28]:
# format and parse dates
df['CREATED_DATE'] = pd.to_datetime(df['CREATED_DATE'])
df['CLOSED_DATE'] = pd.to_datetime(df['CLOSED_DATE'])
df['year'] = df['CREATED_DATE'].dt.year
df['month'] = df['CREATED_DATE'].dt.month
df['date'] = df['CREATED_DATE'].dt.date
df['season'] = df.apply(lambda row: get_season(row['month'], row['year']), axis=1)
df.head()

,SR_NUMBER,SR_SHORT_CODE,CREATED_DATE,STATUS,ORIGIN,CLOSED_DATE,SR_TYPE,STREET_ADDRESS,COMMUNITY_AREA,WARD,LATITUDE,LONGITUDE,year,month,date,season
0,SR19-02929866,SWSNOREM,2019-11-12 09:31:20,Completed,Internet,2019-11-21 09:15:03,Snow – Uncleared Sidewalk Complaint,NaN,NaN,NaN,NaN,NaN,2019,11,2019-11-12,2019-2020
1,SR20-05649092,SWSNOREM,2020-12-17 13:24:37,Completed,Phone Call,2020-12-18 15:12:51,Snow – Uncleared Sidewalk Complaint,5200 S BLACKSTONE AVE,41,4,41.800909458,-87.590514593,2020,12,2020-12-17,2020-2021
2,SR21-00001364,SWSNOREM,2021-01-01 11:40:33,Completed,Mobile Device,2021-01-04 14:33:44,Snow – Uncleared Sidewalk Complaint,66 E CHESTNUT ST,8,42,41.89839859,-87.625722581,2021,1,2021-01-01,2020-2021
3,SR21-00000774,SWSNOREM,2021-01-01 09:11:15,Completed,Internet,2021-01-06 13:37:58,Snow – Uncleared Sidewalk Complaint,2648 N WHIPPLE ST,22,32,41.930232612,-87.703705561,2021,1,2021-01-01,2020-2021
4,SR20-05723378,SWSNOREM,2020-12-31 07:19:37,Completed,Mobile Device,2021-01-08 15:30:57,Snow – Uncleared Sidewalk Complaint,5015 N SPRINGFIELD AVE,14,39,41.972438885,-87.725429908,2020,12,2020-12-31,2020-2021


### look at just current year

In [32]:
df['CREATED_DATE'] = pd.to_datetime(df['CREATED_DATE'])
df = df[(df['season']=='2023-2024')]
len(df)

47

### read community names

In [34]:
# retrieved on 1/11/24, but 2020 Census Population figures should be static

df_population = pd.read_csv("../data/population_cmap_2022.csv")

In [35]:
# simplify dataframe to get only essentials
df_population = df_population[['GEOID','GEOG']]
df_population = df_population.rename(columns={'GEOG':'COMMUNITY_NAME'})
df_population.head()

,GEOID,COMMUNITY_NAME
0,14,Albany Park
1,57,Archer Heights
2,34,Armour Square
3,70,Ashburn
4,71,Auburn Gresham


### remove invalid addresses

In [38]:
df_valid_address = df[(df['STREET_ADDRESS'].isna()==False)
                     & (df['WARD'].isna()==False)
                     & (df['COMMUNITY_AREA'].isna()==False)]
len(df_valid_address)

47

In [39]:
# then correct object types to integer
df_valid_address['COMMUNITY_AREA'] = df_valid_address['COMMUNITY_AREA'].astype(int)
df_valid_address['WARD'] = df_valid_address['WARD'].astype(int)

### merge and save

In [40]:
df_clean = pd.merge(left=df_valid_address,right=df_population,left_on='COMMUNITY_AREA',right_on='GEOID')

In [41]:
df_clean.to_csv("../data/complaints-2023-24.csv")

### review data

In [48]:
df_clean.groupby('COMMUNITY_NAME').size().reset_index(name='complaints').sort_values(by='complaints',ascending=False)

,COMMUNITY_NAME,complaints
10,North Center,7
11,North Lawndale,6
12,Norwood Park,4
6,Humboldt Park,4
9,Logan Square,2
19,West Lawn,2
17,West Elsdon,2
13,Portage Park,2
0,Ashburn,2
8,Lake View,2
